<font size="5" color="#225e22">Importing necessary modules.</font>

In [1]:
import re
import pandas as pd;
import numpy as np
import csv
import collections
import math
import pprint
from matplotlib import pyplot as plt
from sklearn import tree


In [2]:
class Node:
    """ Node class for a decision tree. """
    def __init__(self, names):
        self.names = names

    def classify(x):
        """ Handled by the subclasses. """
        return None

    def dump(self, indent):
        """ Handled by the subclasses. """
        return None


class Leaf(Node):
    def __init__(self, names, value):
        Node.__init__(self, names)
#         print("leaf"+str(names))
        self.value = value

    def classify(self, x):
        return self.value

    def dump(self, indent):
        print(' %d' % self.value)
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)


class Split(Node):
    def __init__(self, names, var, left, right):
        Node.__init__(self, names)
        self.var = var
        self.left = left
        self.right = right
        
    def classify(self, x):
        if x[self.var] == 0:
            return self.left.classify(x)
        else:
            return self.right.classify(x)
      
    def dump(self, indent):
        if indent > 0:
            print('')
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 0 :' % self.names[self.var],end='')
        self.left.dump(indent+1)
        for i in range(0, indent):
            print('| ', end='')
        print('%s = 1 :' % self.names[self.var],end='')
        self.right.dump(indent+1)


Helper function computes entropy of Bernoulli distribution with parameter p

<font size="5" color="#225e22">Entropy function.</font>

In [3]:
def entropy(p,total):
    if total == 0:
        entropy_value = 0
    else:  
        positve_classes = p                   #postive classes in the features i.e number of 1's
        negative_classes = total - p          #postive classes in the features i.e number of 1's
        
        #if feature has only postive examples
        if positve_classes == 0 :
            entropy_value = - 0 - ((negative_classes/total)*(np.log2(negative_classes/total)))
        #if feature has only negative examples
        elif negative_classes==0 :
            entropy_value = - ((positve_classes/total)*(np.log2(positve_classes/total))) - 0
        #entropy calculation for postive examples and negative exmaples
        else:
            entropy_value = - ((positve_classes/total)*(np.log2(positve_classes/total))) - ((negative_classes/total)*(np.log2(negative_classes/total)))
    return entropy_value;


Compute information gain for a particular split, given the counts 

py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n

pxi : number of occurrences of x_i=1

py : number of ocurrences of y=1


<font size="5" color="#225e22">Information Gain function.</font>

In [4]:
def infogain(py_pxi, pxi, py, total):

    Entropy_S = entropy(py,total)   #calculating entropy for a target class
        
    information_gain_x = Entropy_S - ((pxi/total)* entropy(py_pxi,pxi))-(((total-pxi)/total)*entropy((py-py_pxi),(total-pxi)))
         
    return information_gain_x;


<font size="5" color="#225e22">Find feature with maximum Information gain.</font>

In [5]:
def find_maximum_infogain(data,varnames):
    infogain_array = []
    varnames_length =len(varnames)
    last_column = varnames[varnames_length - 1]
    py_pos=collections.Counter(data[last_column])
    py = py_pos[1]                          #py : number of ocurrences of y=1
    for i in varnames:
        py_pxi = 0
        if i is not last_column:
            pxi_pos = collections.Counter(data[i])
            pxi= pxi_pos[1]                 #pxi : number of occurrences of x_i=1
            for x,y in zip(data[i],data[last_column]):
                if(x==1 & y==1):
                    py_pxi += 1            #py_pxi : number of occurences of y=1 with x_i=1 for all i=1 to n
            total=len(data[i])
            infogain_array.append({'feature':i,"gain":infogain(py_pxi, pxi, py, total)})  #Information gain of all features
    maximum_infogain =max(infogain_array, key=lambda x: x['gain'])                        #finding feature with maximum gain 
    return maximum_infogain['feature']
    

<font size="5" color="#225e22">Read data from a file.</font>

In [6]:
# Load data from a file
def read_data(filename):
    f = open(filename, 'r')
    p = re.compile(',')
    data = []
    header = f.readline().strip()
    varnames = p.split(header)
    df = pd.read_csv(filename, usecols=varnames) #dataframe 
    return (df,varnames)

OTHER SUGGESTED HELPER FUNCTIONS:

-collect counts for each variable value with each class label

-find the best variable to split on, according to mutual information

-partition data based on a given variable	
	


Build tree in a top-down manner, selecting splits until we hit a pure leaf or all splits look bad.


In [7]:
def build_tree(data, varnames):
    target_column = varnames[len(varnames)-1]           #fetching target class
    node=find_maximum_infogain(data, varnames)          # node with maximum gain
    feature_attributes = np.unique(data[node])          #unique values of a node, [0,1]
    for attributes in feature_attributes:
        subdata = data.loc[data[node] == attributes]        #
        targetvalue,targetcounts = np.unique(subdata[target_column], return_counts=True) 
        if attributes == 0:
            if(len(targetvalue)) == 1:
                l1 = Leaf(varnames,targetvalue[0])
            else:
                l1 = build_tree(subdata,varnames)
        else:
            if(len(targetvalue)) == 1:
                r1 = Leaf(varnames,targetvalue[0])
            else:
                r1 = build_tree(subdata,varnames)
    root = Split(varnames, data.columns.get_loc(node) , l1, r1)
    return root



In [8]:
(train, varnames) = read_data('agaricuslepiotatrain1.csv')
(test, testvarnames) = read_data('agaricuslepiotatest1.csv') 

Here we load data.
Each example is a list of attribute values, where the last element in the list is the class value.


In [9]:
pd.DataFrame(train).head()

,cap-shape-bell,cap-shape-conical,cap-shape-convex,cap-shape-flat,cap-shape-knobbed,cap-shape-sunken,cap-surface-fibrous,cap-surface-grooves,cap-surface-scaly,cap-surface-smooth,...,population-several,population-solitary,habitat-grasses,habitat-leaves,habitat-meadows,habitat-paths,habitat-urban,habitat-waste,habitat-woods,poisonous
0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1
4,0,0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [10]:
varnames

['cap-shape-bell',
 'cap-shape-conical',
 'cap-shape-convex',
 'cap-shape-flat',
 'cap-shape-knobbed',
 'cap-shape-sunken',
 'cap-surface-fibrous',
 'cap-surface-grooves',
 'cap-surface-scaly',
 'cap-surface-smooth',
 'cap-color-brown',
 'cap-color-buff',
 'cap-color-cinnamon',
 'cap-color-gray',
 'cap-color-green',
 'cap-color-pink',
 'cap-color-purple',
 'cap-color-red',
 'cap-color-white',
 'cap-color-yellow',
 'bruises?-bruises',
 'odor-almond',
 'odor-anise',
 'odor-creosote',
 'odor-fishy',
 'odor-foul',
 'odor-musty',
 'odor-none',
 'odor-pungent',
 'odor-spicy',
 'gill-attachment-attached',
 'gill-attachment-descending',
 'gill-attachment-free',
 'gill-attachment-notched',
 'gill-spacing-close',
 'gill-spacing-crowded',
 'gill-spacing-distant',
 'gill-size-broad',
 'gill-color-black',
 'gill-color-brown',
 'gill-color-buff',
 'gill-color-chocolate',
 'gill-color-gray',
 'gill-color-green',
 'gill-color-orange',
 'gill-color-pink',
 'gill-color-purple',
 'gill-color-red',
 'gill

<font size="5" color="#225e22">Build the decision tree.</font>

In [11]:
root = build_tree(train, varnames)

In [12]:
root.dump(0)

odor-foul = 0 :
| gill-size-broad = 0 :
| | odor-none = 0 :
| | | gill-spacing-close = 0 :
| | | | bruises?-bruises = 0 : 1
| | | | bruises?-bruises = 1 : 0
| | | gill-spacing-close = 1 : 1
| | odor-none = 1 :
| | | stalk-surface-above-ring-silky = 0 :
| | | | bruises?-bruises = 0 : 0
| | | | bruises?-bruises = 1 : 1
| | | stalk-surface-above-ring-silky = 1 : 1
| gill-size-broad = 1 :
| | spore-print-color-green = 0 : 0
| | spore-print-color-green = 1 : 1
odor-foul = 1 : 1


<font size="5" color="#225e22">Calcuating the accuracy.</font>

In [13]:
def accuracy(data):
    correct = 0
    # The position of the class label is the last element in the list.
    train_data = data.to_numpy()
#     print(test)
    yi = len(train_data[0]) - 1
    for x in train_data:
    # Classification is done recursively by the node class.
    # This should work as-is.
        pred = root.classify(x)
        if pred == x[yi]:
            correct += 1
        acc = float(correct)/len(train_data)
    return acc;
   

In [14]:
 print("Train Accuracy: {}".format(accuracy(train)))

Train Accuracy: 1.0


In [15]:
 print("Test Accuracy: {}".format(accuracy(test)))

Test Accuracy: 0.9792941176470589
